<p align="center"><img width="50%" src="https://www.modelshare.ai/img/ModelShareLogo_COLOR.abf79305.png" /></p>


---




<p align="center"><h1 align="center">Quick Start: Titanic Tabular Classification Tutorial</h1> 


## **Create Model Playground ML Project Space: Submit, Track, and Deploy ML Model(s) in about 5 mins**
> ### (1) Set Up Environment 
> ### (2) Create a Model Playground
> ### (3) Submit, Track, and Deploy ML Model(s)
> ### (4) Improve Models and Analyze Model Performance
> ### (5) Maintain Your Model Playground

## **(1) Set Up Environment**

In order to deploy a Model Share AI Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your Model Share AI username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://aimodelshare.readthedocs.io/en/latest/create_credentials.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github. Model Share AI does not store your credentials.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file
from aimodelshare.aws import configure_credentials 
configure_credentials()

In [ ]:
# Set credentials 
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model")

Modelshare.ai login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# Download Titanic tabular data
from aimodelshare.data_sharing.download_data import import_quickstart_data
X_train, X_test, y_train_labels, y_test, example_data, y_test_labels = import_quickstart_data("titanic")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(2) Create a Model Playground**

### **Create a Model Playground**


> ### Each Model Playground is a unique ML project space with lots of useful functionality...
- Use playground.submit_model() to track ML models in experiments and/or competition leaderboards
- Save, version, and reuse ML models each time they are submitted
- Use playground.compare_models() to compare model architectures with git like colorized model diffs
- Use playground.deploy_model() to deploy models instantly into live REST APIs and/or web-apps.  

*  *First, you need to intantiate a ModelPlayground() object locally. Please specify the data type of your learning task (e.g., image, tabular, etc), whether the model is a classification or a regression model, and whether your playground should be public or private. If you choose "private" only approved team members can interact your model playground*  
*  *Second, you can to create a playground page on the modelshare.ai website. You need to submit evaluation data that submitted models can be evaluated against.*  



In [ ]:
# Instantiate Model Playground object
from aimodelshare.playground import ModelPlayground
myplayground=ModelPlayground(input_type="tabular", 
                             task_type="classification", 
                             private=False)

# Create Model Playground Page on modelshare.ai website
myplayground.create(eval_data=y_test_labels)

Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 55 seconds. 
 Playground Url: "https://iun27fv8t5.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic tab.
https://www.modelshare.ai/detail/model:3527

Check out your Model Playground page for more.


##  **3) Submit, Track, and Deploy Model(s)**


### **Write a Preprocessor Function**


> ###   Preprocessing functions play a crucial role in preparing data for your model by converting it into the specific format that is needed to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for an image filepath and values to reshape the image height and width.*  



In [ ]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently
# In this example we use Sklearn's Column transformer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Preprocess data using sklearn's Column Transformer approach

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #'imputer' names the step
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']

# Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [ ]:
# Here is where we actually write the preprocessor function:
def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

### **Train Your Machine Learning Model**


> ### Model Share supports Scikit-Learn, Tensorflow,  Keras, and Pytorch models out of the box.  

*  *Train your model using your favorite ML library. Here, we are training models with Sklearn.*  
*  *Use your model to make predictions on the test set.*  




In [ ]:
# Train model 1: Logistic Regression with L1 Regularization (Lasso)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, penalty='l1', solver = 'liblinear')
model.fit(preprocessor(X_train), y_train_labels) # Fitting to the training set.
model.score(preprocessor(X_train), y_train_labels) # Fit score, 0-1 scale.

0.7812798471824259

In [ ]:
#-- Generate predicted values (a list of predicted labels, in this case either "survived" or "died") 
prediction_labels = model.predict(preprocessor(X_test))

### **Submit a Model to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  




In [ ]:
# Submit Model to Experiment Leaderboard
# (Option: set submission_type="competition" to submit to competition leaderboard)

myplayground.submit_model(model=model,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 1.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3527


### **Deploy a Model to a Prediction REST API and Web-app**


> ### Update the prediction API behind your Model Playground with a new model and verify the model performance metrics in your Model Playground.

Use playground.deploy_model() with the following key arguments:
* *model_version: Simply refer to the model by its leaderboard version number.*
* *example_data: deploy_model builds a web-app for you.  It needs some example data for end users to download to try your web-app.  For tabular data provide a short pandas DataFrame in the format expected by your preprocessor.* 
*  *y_train: For classification problems deploy_model needs well ordered y_training data labels.  Provide a python list of y_train labels or a one hot encoded Pandas y_training data frame, so your web-app and prediction api return the correct prediction labels*

In [ ]:
# Deploy model by version number
myplayground.deploy_model(model_version=1, 
                          example_data=example_data, 
                          y_train=y_train_labels)

Runtime model & preprocessor for api: https://iun27fv8t5.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 1.

Model metrics are now updated and verified for this model playground.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3527


## **(4) Improve Models and Analyze Model Performance**

### **Set Submission Credentials**

> ### Authorized users can submit new models after setting credentials using modelshare.ai username/password.  

*  *This only needs to be done if you want to submit models in a new session.*  
*  *skip this step if you have already set your deployment credentials.*  




In [ ]:
# example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m
apiurl=myplayground.playground_url 

set_credentials(apiurl=apiurl)

Modelshare.ai Username:··········
Modelshare.ai Password:··········
Modelshare.ai login credentials set successfully.


### **Submit More Models to your Model Playground**


> ### You can submit models to your Model Playground page in order to evaluate them, extract model metadata, compare them to other models, and turn them into REST API endpoints.  

*  *Pass your model to the submit model function. You can either pass a model object, an ONNX object, or a filepath to an ONNX file.*  
*  *Pass your preprocessor to ensure that your model can process new data.*
*  *Pass your predictions to be evaluated against your playground evaluation data.*  


In [ ]:
# Create model 2 (L2 Regularization - Ridge)
from sklearn.linear_model import LogisticRegression

model_2 = LogisticRegression(C=.01, penalty='l2')
model_2.fit(preprocessor(X_train), y_train_labels) # Fitting to the training set.
model_2.score(preprocessor(X_train), y_train_labels) # Fit score, 0-1 scale.

0.7860553963705826

In [ ]:
# Submit Model 2

# Generate predicted y values (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Experiment Leaderboard
myplayground.submit_model(model = model_2,
                          preprocessor=preprocessor,
                          prediction_submission=prediction_labels)


Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 
Your model has been submitted to experiment as model version 2.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3527


### **Check Experiment Leaderboard**


> ### Leaderboards allow you to compare model architectures ranked by predictive performance.  

*  *Query leaderboard data for models submitted to your Model Playground page.* 
*  *You can either stylize the leaderboard or use pandas to run your own analyses to explore which model architectures perform well.*  




In [ ]:
# Check experiment leaderboard
data = myplayground.get_leaderboard()
myplayground.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,model_type,num_params,optimizer,username,version
0,80.15%,78.00%,77.31%,79.11%,sklearn,LogisticRegression,10,liblinear,gstreett,1
1,80.15%,76.61%,77.46%,75.96%,sklearn,LogisticRegression,10,lbfgs,gstreett,2


### **Compare Two or More Models**


> ### Model comparisons can help you to identify differences in model architecture at a glance.  

*  *Simply pass the version numbers of the models you would like to compare.* 
*  *For Keras and Pytoch models you can compare architectures through color-coded layer types. For Scikit-Learn models, you can compare hyperparameter settings.*  

In [ ]:
# Compare two or more models
data=myplayground.compare_models([1,2], verbose=1)
myplayground.stylize_compare(data)

,param_name,default_value,model_version_1,model_version_2
0,C,1.000000,10,0.010000
1,class_weight,None,None,None
2,dual,False,False,False
3,fit_intercept,True,True,True
4,intercept_scaling,1,1,1
5,l1_ratio,None,None,None
6,max_iter,100,100,100
7,multi_class,auto,auto,auto
8,n_jobs,None,None,None
9,penalty,l2,l1,l2


### **Check the Structure of Evaluation Data**

> ### Your team needs to understand the format of the evaluation data, to ensure that their submissions can be processed.  

*  *Make sure class labels, data types, and dimensions of predictions correspond to submitted evaluation data.*
*  *This is important to generate evaluation metrics for your submission.*


In [ ]:
# Check structure of evaluation data
myplayground.inspect_eval_data()

{'ytest_example': ['died', 'died', 'survived', 'died', 'died'],
 'y_length': 262,
 'class_labels': ['died', 'survived'],
 'class_balance': {'died': 178, 'survived': 84},
 'label_dtypes': {"<class 'str'>": 262}}

## **(5) Maintaining Your Model Playground**

### **Update the Runtime Model (the model that is used for predictions in your REST API/Web-App**


> ### You can update the runtime model anytime by picking one of the previously submitted models.  

* *Simply refer to the model by its leaderboard version number.*
* *You can choos a model from the Competition or Experiment leaderboard by specifying the submission_type argument.*

In [ ]:
# Update runtime model
myplayground.update_runtime_model(model_version=2)

Runtime model & preprocessor for api: https://iun27fv8t5.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.

Visit your Model Playground Page for more.
https://www.modelshare.ai/detail/model:3527


### **Delete Deployment**


> ### You can delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models.  

* *This is final - there is no way to recover your Model Playground if you delete it.*
* *We have commented the line for safety reasons.*

In [ ]:
#myplayground.delete_deployment()